In [1]:
using ITensors.NDTensors
using ITensors
using LinearAlgebra
using PyPlot
using BenchmarkTools

### Extension of the qr decomposition of ITensors to Blocksparse matrices and therefore to QNITensors

In [6]:
function qr_small(T::BlockSparseTensor{ElT,2,StoreT,IndsT}; kwargs...) where{ElT, StoreT,IndsT}
    Qs = Vector{DenseTensor{ElT,2}}(undef, nnzblocks(T))
    Rs = Vector{DenseTensor{ElT,2}}(undef, nnzblocks(T))
    
    for (jj,b) in enumerate(eachnzblock(T))
        blockT = blockview(T,b)
        QRb = qr(blockT; kwargs...)
        
        if(isnothing(QRb))
            return nothing
        end
        
        Q, R   = QRb
        Qs[jj] = Q
        Rs[jj] = R
        
    end   
    
    # getting total number of blocks
    nnzblocksT = nnzblocks(T)
    nzblocksT  = nzblocks(T)
    
    nb1_lt_nb2 = (
             nblocks(T)[1] < nblocks(T)[2] ||
            (nblocks(T)[1] == nblocks(T)[2] && dim(T, 1) < dim(T, 2))
          )

    # setting the right index of the Q isometry, this should be
    # the smaller index of the two indices of of T
    qindl = ind(T,1)
    if nb1_lt_nb2
        qindr = sim(ind(T, 1))
    else
        qindr = sim(ind(T, 2))
    end
    
    # can qindr have more blocks than T?
    if nblocks(qindr) > nnzblocksT
        resize!(qindr, nnzblocksT)
    end
    
    for n in 1:nnzblocksT
        q_dim_red = minimum(dims(Rs[n]))
        NDTensors.setblockdim!(qindr, q_dim_red, n)
    end
    
    # correcting the direction of the arrow
    # since qind2r is basically a copy of qind1r
    # if one have to be corrected the other one 
    # should also be corrected
    if(dir(qindr) != dir(qindl))
        qindr = dag(qindr)
    end
    
    indsQ = setindex(inds(T), dag(qindr), 2)
    
    # R left index
    rindl = qindr
    rindr = ind(T,2)
    
    #if(dir(rindl) != dir(rindr))
    #    rindl = dag(rindl)
    #end
    
    indsR = setindex(inds(T), qindr, 1)
    
    nzblocksQ = Vector{Block{2}}(undef, nnzblocksT)
    nzblocksR = Vector{Block{2}}(undef, nnzblocksT)
    
    for n in 1:nnzblocksT
        blockT = nzblocksT[n]
        
        blockQ = (blockT[1], UInt(n))
        nzblocksQ[n] = blockQ
       
        blockR = (blockT[2], UInt(n))
        nzblocksR[n] = blockR
    end
    
    Q = BlockSparseTensor(ElT, undef, nzblocksQ, indsQ)
    R = BlockSparseTensor(ElT, undef, nzblocksR, indsR)
    
    
    for n in 1:nnzblocksT
        Qb, Rb = Qs[n], Rs[n]
        blockQ = nzblocksQ[n]
        blockR = nzblocksR[n]
        
         if VERSION < v"1.5"
            # In v1.3 and v1.4 of Julia, Ub has
            # a very complicated view wrapper that
            # can't be handled efficiently
            Qb = copy(Qb)
            Rb  = copy(Vb)
        end
        
        blockview(Q, blockQ) .= Qb
        blockview(R, blockR) .= Rb
    end
    
    # correcting the fluxes of the 
    # two tensors, such that 
    # Q has 0 flux for all blocks
    # and R has the total flux of the system
    for b in nzblocks(Q)
        i1 = inds(Q)[1]
        i2 = inds(Q)[2]
        r1 = inds(R)[1]
        newqn = -dir(i2) * flux(i1 => Block(b[1]))
        ITensors.setblockqn!(i2, newqn, b[2])
        ITensors.setblockqn!(r1, newqn, b[2])
    end
    
    return Q, R
end

function qr_small(T::BlockSparseTensor{ElT,1,StoreT,IndsT}; kwargs...) where{ElT,StoreT,IndsT}
    Qs = Vector{DenseTensor{ElT,2}}(undef, nnzblocks(T))
    Rs = Vector{DenseTensor{ElT,1}}(undef, nnzblocks(T))

    for (jj,b) in enumerate(eachnzblock(T))
        blockT = blockview(T,b)
        blockT = reshape(blockT, dim(blockT), 1)
        QRb = qr(blockT; kwargs...)
        
        if(isnothing(QRb))
            return nothing
        end
        
        Q, R   = QRb
        Qs[jj] = Q
        Rs[jj] = reshape(R,dim(R))
    end   
    
    # getting total number of blocks
    nnzblocksT = nnzblocks(T)
    nzblocksT  = nzblocks(T)

    # setting the right index of the Q isometry, this should be
    # the smaller index of the two indices of of T
    qindl = ind(T,1)
    qindr = sim(qindl)
    
    # can qindr have more blocks than T?
    if nblocks(qindr) > nnzblocksT
        resize!(qindr, nnzblocksT)
    end
    
    for n in 1:nnzblocksT
        q_dim_red = minimum(dims(Rs[n]))
        NDTensors.setblockdim!(qindr, q_dim_red, n)
    end
    
    # correcting the direction of the arrow
    # since qind2r is basically a copy of qind1r
    # if one have to be corrected the other one 
    # should also be corrected
    if(dir(qindr) != dir(qindl))
        qindr = dag(qindr)
    end
    
    indsQ = (qindl, dag(qindr))
    
    # R left index
    rindl = qindr
    #rindr = ind(T,2)
    
    #if(dir(rindl) != dir(rindr))
    #    rindl = dag(rindl)
    #end
    
    indsR = setindex(inds(T)[1:1], rindl, 1)
    
    nzblocksQ = Vector{Block{2}}(undef, nnzblocksT)
    nzblocksR = Vector{Block{1}}(undef, nnzblocksT)
    
    for n in 1:nnzblocksT
        blockT = nzblocksT[n]
        
        blockQ = (blockT[1], UInt(n))
        nzblocksQ[n] = blockQ
       
        blockR = (UInt(n), )
        nzblocksR[n] = blockR
    end
    
    Q = BlockSparseTensor(ElT, undef, nzblocksQ, indsQ)
    R = BlockSparseTensor(ElT, undef, nzblocksR, indsR)
    
    
    for n in 1:nnzblocksT
        Qb, Rb = Qs[n], Rs[n]
        blockQ = nzblocksQ[n]
        blockR = nzblocksR[n]
        
         if VERSION < v"1.5"
            # In v1.3 and v1.4 of Julia, Ub has
            # a very complicated view wrapper that
            # can't be handled efficiently
            Qb = copy(Qb)
            Rb  = copy(Vb)
        end
        
        blockview(Q, blockQ) .= Qb
        blockview(R, blockR) .= Rb
    end
    
    # correcting the fluxes of the 
    # two tensors, such that 
    # Q has 0 flux for all blocks
    # and R has the total flux of the system
    for b in nzblocks(Q)
        i1 = inds(Q)[1]
        i2 = inds(Q)[2]
        r1 = inds(R)[1]
        newqn = -dir(i2) * flux(i1 => Block(b[1]))
        ITensors.setblockqn!(i2, newqn, b[2])
        ITensors.setblockqn!(r1, newqn, b[2])
    end
    
    return Q, R
end


function qr_small(A::ITensor, Linds...; kwargs...)
    tags::TagSet = get(kwargs, :tags, "Link,qr")
    
    Lis = commoninds(A, IndexSet(Linds...))
    Ris = uniqueinds(A, Lis)
    
    if length(Lis) == 0 && length(Ris) == 0
        error(
                "In `qr`, the left and right indices are empty (the indices of `A` are ($(inds(A))), but the input indices are ($Lis)). For now, this is not supported. You may have accidentally input the wrong indices.",
            )
    end
    
    
    CL = combiner(Lis...)
    CR = combiner(Ris...)
    
    AC = A * CR * CL
    
    if(length(Lis) == 0)
        At = tensor(AC)
        # build trivial leg
        qtriv = zero(qn(inds(AC)[1], Block(1)))
        qidx  = Index(qtriv => 1; tags = tags, dir = ITensors.In)
        Q  = delta(qidx)
        
        nzblocksR = Vector{Block{2}}(undef, nnzblocks(AC))
        for (jj,b) in enumerate(eachnzblock(At))
            blockR = (UInt(1), b[1])
            nzblocksR[jj] = blockR
        end
        Rt = BlockSparseTensor(eltype(At), undef, nzblocksR, (dag(qidx),inds(AC)...))
        for (jj,b) in enumerate(eachnzblock(At))
            ACB = blockview(At,b)
            if VERSION < v"1.5"
                # In v1.3 and v1.4 of Julia, Ub has
                # a very complicated view wrapper that
                # can't be handled efficiently
                ACB = copy(ACB)
            end 
            ACB = reshape(ACB,(1,dim(ACB)))
            blockR = nzblocksR[jj]
            blockview(Rt, blockR) .= ACB
        end
        RC = itensor(Rt)
        
        R = RC*dag(CR)
        return Q,R, qidx
    elseif(length(Ris) == 0)
        QRT = qr_small(tensor(AC); kwargs...)
    else
        cL = combinedind(CL)
        cR = combinedind(CR)
    
        if inds(AC) != IndexSet(cL, cR)
            AC = permute(AC, cL, cR)
        end
    
        QRT = qr_small(tensor(AC); kwargs...)
    end
    
    
    if isnothing(QRT)
        return nothing
    end
    QT, RT = QRT
    QC, RC = itensor(QT), itensor(RT)
    
    qidx  = commonind(QC,RC)
    
    
    Q = QC * dag(CL)
    R = RC * dag(CR)
    
    settags!(Q, tags, qidx)
    settags!(R,  tags, qidx)
    qidx = settags(qidx, tags)
    
    return Q, R, qidx
end

qr_small (generic function with 3 methods)

### Requesting a full qr decomposition for dense tensors

In [7]:
function qr_full(T::DenseTensor{ElT,2,IndsT}; kwargs...) where{ElT,IndsT}
    positive = get(kwargs, :positive, false)
    if(positive)
        error("Not implemented")
    else
        #QM, RM = qr(matrix(T))
        F = qr(matrix(T))
    end
    
    # converting QM to Q1 and Q2
    # getting the correct dimensions
    dim1  = dim(T,1)
    dim2  = dim(T,2)
    q_dim = size(F.Q,1)
    q_dim_red = min(dim1,dim2)
    
    # getting the new index objects with correct dimensions.
    # the right index of the orthogonal Q2 part will be added a `ortho` tag
    q,r = inds(T)
    
    q_red   = eltype(IndsT)(q_dim_red)
    q_ortho = eltype(IndsT)(q_dim - q_dim_red)
    #q_red   = Index(q_dim_red, tags(q))
    #q_ortho = Index(q_dim - q_dim_red, tags(q))
    #q_ortho = addtags(q_ortho,"ortho")
    
    # Indecies of the Q1,Q2 and R Tensors
    Q1inds = IndsT((ind(T, 1), q_red))
    Q2inds = IndsT((ind(T, 1), q_ortho))
    Rinds  = IndsT((q_red, ind(T, 2)))
    
    # reading out the matrices
    QM = F.Q*Matrix{ElT}(I,size(F.Q))
    #Q1M = @view QM[:,1:q_dim_red]
    #Q2M = @view QM[:,q_dim_red+1:end]
    
    Q1 = tensor(Dense(vec(QM[:,1:q_dim_red])), Q1inds)
    Q2 = tensor(Dense(vec(QM[:,q_dim_red+1:end])), Q2inds)
    #RM = F.R
    R  = tensor(Dense(vec(F.R)), Rinds)
    return Q1,Q2,R
end

function qr_full(
    T::DenseTensor{<:Number,N,IndsT}, Lpos::NTuple{NL,Int}, Rpos::NTuple{NR,Int}; kwargs...
    ) where{N,IndsT,NL,NR}
    
    
    M = NDTensors.permute_reshape(T,Lpos,Rpos)
    Q1M, Q2M, RM  = qr_full(M; kwargs...)
    qn = ind(Q1M,2)
    qo = ind(Q2M,2)
    r  = ind(RM,1)
    Linds = NDTensors.similartype(IndsT, Val{NL})(ntuple(i -> inds(T)[Lpos[i]], Val(NL)))
    Q1inds = NDTensors.push(Linds,qn)
    Q2inds = NDTensors.push(Linds,qo)
    Q1 = reshape(Q1M,Q1inds)
    Q2 = reshape(Q2M,Q2inds)
    Rinds = NDTensors.similartype(IndsT, Val{NR})(ntuple(i -> inds(T)[Rpos[i]], Val(NR)))
    Rinds = NDTensors.pushfirst(Rinds,r)
    R = reshape(RM,Rinds)
    return Q1, Q2, R
end



function qr_full_dense(A::ITensor, Linds...; kwargs...)
    tags::TagSet = get(kwargs, :tags, "Link,qr")
    Lis = commoninds(A, IndexSet(Linds...))
    Ris = uniqueinds(A,Lis)
    
    Lpos, Rpos = NDTensors.getperms(inds(A), Lis, Ris)
    Q1T, Q2T, RT = qr_full(tensor(A), Lpos, Rpos; kwargs...)
    Q1, Q2, R = itensor(Q1T), itensor(Q2T), itensor(RT)
    qidx = commonind(Q1,R)
    lidx = uniqueinds(Q1,qidx)
    settags!(Q1,tags,qidx) 
    settags!(R ,tags,qidx)
    qidx = settags(qidx,tags)

    tags = ITensors.addtags(tags,"ortho")
    qoidx = uniqueinds(Q2,lidx)
    settags!(Q2,tags,qoidx)
    return Q1, Q2, R, qidx,qoidx
end


qr_full_dense (generic function with 1 method)

### Requesting a full qr decomposition for block sparse (QN ITensors) tensors

In [49]:
function qr_full(T::BlockSparseTensor{ElT,1,StoreT, IndsT}; kwargs...) where{ElT, StoreT, IndsT}
    # blocks of the tensor T and number of blocks
    nnzblocksT = nnzblocks(T)
    nzblocksT  = nzblocks(T)
    
    # setting the right index of the Q isometry, this should be
    # the smaller index of the two indices of of T
    qindl  = ind(T,1)
    qind1r = sim(qindl)
    
    
    if nblocks(qind1r) > nnzblocksT
        resize!(qind1r, nnzblocksT)
    end
    
    # correcting the direction of the arrow
    # since qind2r is basically a copy of qind1r
    # if one have to be corrected the other one 
    # should also be corrected
    if(dir(qind1r) == dir(qindl))
        qind1r = dag(qind1r)
    end
    
    # qr decomposition of the different original blocks
    Q1s = Vector{DenseTensor{ElT,2}}(undef, nnzblocksT)
    Q2s = Vector{DenseTensor{ElT,2}}(undef, nnzblocksT)
    Rs  = Vector{DenseTensor{ElT,1}}(undef, nnzblocksT)
    
    
    for (jj,b) in enumerate(eachnzblock(T))
        blockT = blockview(T,b)
        blockT = reshape(blockT, dim(blockT), 1)
        
        QRb    = qr_full(blockT)
        
        if(isnothing(QRb))
            return nothing
        end
        
        Q1,Q2,R = QRb
        Q1s[jj] = Q1
        Q2s[jj] = Q2
        R = reshape(R, dim(R))
        Rs[jj]  = R
    end
    
    for n in 1:nnzblocksT
        q_dim_red = minimum(dims(Rs[n]))
        NDTensors.setblockdim!(qind1r, q_dim_red, n)
    end
    
    indsQ1 = (qindl, qind1r)
    
    
    # R left index
    rindl = dag(qind1r)
    indsR = setindex(inds(T)[1:1], rindl, 1)

    nzblocksQ1 = Vector{Block{2}}(undef, nnzblocksT)
    nzblocksR  = Vector{Block{1}}(undef, nnzblocksT)

    for n in 1:nnzblocksT
        blockT = nzblocksT[n]
        
        blockQ = (blockT[1], UInt(n))
        nzblocksQ1[n] = blockQ
    
        blockR = (UInt(n), )
        nzblocksR[n] = blockR
    end
    
    Q1 = BlockSparseTensor(ElT, undef, nzblocksQ1, indsQ1)
    R  = BlockSparseTensor(ElT, undef, nzblocksR, indsR)

    
    for n in 1:nnzblocksT
        Q1b, Rb = Q1s[n], Rs[n]
        blockQ1 = nzblocksQ1[n]
        blockR  = nzblocksR[n]
        
        if VERSION < v"1.5"
            # In v1.3 and v1.4 of Julia, Ub has
            # a very complicated view wrapper that
            # can't be handled efficiently
            Q1b = copy(Q1b)
            Rb  = copy(Vb) 
        end
        
        blockview(Q1, blockQ1) .= Q1b
        blockview(R,  blockR)  .= Rb
    end
    
    
    # correcting the fluxes of the 
    # two tensors, such that 
    # Q has 0 flux for all blocks
    # and R has the total flux of the system
    for b in nzblocks(Q1)
        i1 = inds(Q1)[1]
        i2 = inds(Q1)[2]
        r1 = inds(R)[1]
        newqn = -dir(i2) * flux(i1 => Block(b[1]))
        ITensors.setblockqn!(i2, newqn, b[2])
        ITensors.setblockqn!(r1, newqn, b[2])
    end
        
    
    #### up to here, Q1 and R are finished builded
    ### Now the next step is to determine 
    ### wether there are possible unadressed qn 
    ### with the correct flux
    
    # the flux of Q1 should be zero through the construction
    # therefore we need just to find possible blocks
    # which would be probl. available in the full (qindl,dag(qindl))
    # matrix with 0 flux but are not present in the Q1 matrix
    # these have to be filled in the Q2 matrix
    
    imgsp = space(qindl)
    q1sp  = space(qind1r)
    q1q   = qn.(q1sp)
    q1d   = blockdim.(q1sp)
    
    orthosp = Vector{Pair{QN,Int}}()
    
    # imgsp is a Vector{Pair{QN, Int}}, such that
    # we can unpack it to get the qn and dimension directly
    exists_in_both = Vector{Int}() # list of tracking blocks existing in both tensors
    for (spq, spd) in imgsp
        # now test it the qn is present in the q1q array
        # holding all target qn numbers of the Q1 tensor
        if(spq in q1q)
            idx = findfirst(x -> x == spq, q1q)
            # check if it has the full dimensionality
            if(q1d[idx] < spd)
                push!(orthosp, spq => spd - q1d[idx])
                push!(exists_in_both,length(orthosp))
            end
        else
            push!(orthosp, spq => spd)
        end
    end
    
    # generating the index for the orthogonal space
    qortags = addtags(tags(qind1r), "ortho")
    qind2r  = Index(orthosp, qortags; dir = dir(qind1r))
    
    # savety check, the dimensionality should add up
    # to the total dimensionality of the left leg
    @assert dim(qind1r) + dim(qind2r) == dim(qindl)
    
    indsQ2 = (qindl, qind2r)
    
    # now the different blocks of the Q2 tensor have to be created
    # their size has to be the same as the number of blocks in the
    # right Index
    nzblocksQ2 = Vector{Block{2}}(undef, nblocks(qind2r))
    
    # go through all possible targets
    for (jj, img) in enumerate(orthosp)
        imgq = qn(img)
        blockQ2L = ITensors.block(first, qindl, imgq)
        blockQ2R = ITensors.block(first, qind2r, dir(qind2r)*imgq)
        nzblocksQ2[jj] = (blockQ2L,blockQ2R)
    end
    
    # now build the Q2 tensor
    Q2 = BlockSparseTensor(ElT, undef, nzblocksQ2, indsQ2)
   
    # now looping through all blocks in the Q2 matrix
    for (jj, blockQ2) in enumerate(nzblocksQ2)
        if(jj in exists_in_both)
            qr = qn(qind2r, blockQ2[2]) # qn of the block
            # getting the block for this qn in the qind1r index
            blockQ1R = ITensors.block(first, qind1r, dir(qind1r)*qr)
            # getting the original index for that transition
            # this encodes the index in the list of dense qr decompositions
            idx = Int(blockQ1R[1])
            Q2b = Q2s[idx]
            
        else    
            blkdim = blockdim(orthosp[jj])
            Q2b = Matrix{ElT}(I,blkdim, blkdim)
        end
        blockview(Q2, blockQ2) .= Q2b
    end
    
    return Q1,Q2,R
end

function qr_full(T::BlockSparseTensor{ElT,2,StoreT,IndsT}; kwargs...) where{ElT, StoreT,IndsT}
    # blocks of the tensor T and number of blocks
    nnzblocksT = nnzblocks(T)
    nzblocksT  = nzblocks(T)


    # getting the large index
    nb1_lt_nb2 = (
             nblocks(T)[1] < nblocks(T)[2] ||
            (nblocks(T)[1] == nblocks(T)[2] && dim(T, 1) ≤ dim(T, 2))
              )

    # setting the right index of the Q isometry, this should be
    # the smaller index of the two indices of of T
    # while for the orthogonal complement we choose the large
    # index as right index
    qindl = ind(T,1)
    if nb1_lt_nb2
        # doing a fast exit here 
        #Q1,R = qr_small(T)
        Q1,R = qr_small(T)
        Q2 = nothing
        return Q1,Q2,R
        #qind1r = sim(ind(T, 1))
        #qind2r = sim(ind(T, 2))
    else
        qind1r    = sim(ind(T, 2))
    end
    
    if nblocks(qind1r) > nnzblocksT
        resize!(qind1r, nnzblocksT)
    end
    
    # correcting the direction of the arrow
    # since qind2r is basically a copy of qind1r
    # if one have to be corrected the other one 
    # should also be corrected
    if(dir(qind1r) == dir(qindl))
        qind1r = dag(qind1r)
    end


    # qr decomposition of the different original blocks
    Q1s = Vector{DenseTensor{ElT,2}}(undef, nnzblocksT)
    Q2s = Vector{DenseTensor{ElT,2}}(undef, nnzblocksT)
    Rs  = Vector{DenseTensor{ElT,2}}(undef, nnzblocksT)

    for (jj,b) in enumerate(eachnzblock(T))
        blockT = blockview(T,b)
        QRb    = qr_full(blockT)
        
        if(isnothing(QRb))
            return nothing
        end
        
        Q1,Q2,R = QRb
        Q1s[jj] = Q1
        Q2s[jj] = Q2
        Rs[jj]  = R
    end

    for n in 1:nnzblocksT
        q_dim_red = minimum(dims(Rs[n]))
        NDTensors.setblockdim!(qind1r, q_dim_red, n)
    end

    indsQ1 = setindex(inds(T), qind1r, 2)
    # R left index
    rindl = dag(qind1r)
    indsR = setindex(inds(T), rindl, 1)

    nzblocksQ1 = Vector{Block{2}}(undef, nnzblocksT)
    nzblocksR  = Vector{Block{2}}(undef, nnzblocksT)

    for n in 1:nnzblocksT
        blockT = nzblocksT[n]
        blockQ = (blockT[1], UInt(n))
        nzblocksQ1[n] = blockQ
    
        blockR = (blockT[2], UInt(n))
        nzblocksR[n] = blockR
    end


    Q1 = BlockSparseTensor(ElT, undef, nzblocksQ1, indsQ1)
    R  = BlockSparseTensor(ElT, undef, nzblocksR, indsR)

    for n in 1:nnzblocksT
        Q1b, Rb = Q1s[n], Rs[n]
        blockQ1 = nzblocksQ1[n]
        blockR  = nzblocksR[n]
        
        if VERSION < v"1.5"
            # In v1.3 and v1.4 of Julia, Ub has
            # a very complicated view wrapper that
            # can't be handled efficiently
            Q1b = copy(Q1b)
            Rb  = copy(Vb) 
        end
        
        blockview(Q1, blockQ1) .= Q1b
        blockview(R,  blockR)  .= Rb
    end
    
        
    # correcting the fluxes of the 
    # two tensors, such that 
    # Q has 0 flux for all blocks
    # and R has the total flux of the system
    for b in nzblocks(Q1)
        i1 = inds(Q1)[1]
        i2 = inds(Q1)[2]
        r1 = inds(R)[1]
        newqn = -dir(i2) * flux(i1 => Block(b[1]))
        ITensors.setblockqn!(i2, newqn, b[2])
        ITensors.setblockqn!(r1, newqn, b[2])
    end
    
    #### up to here, Q1 and R are finished builded
    ### Now the next step is to determine 
    ### wether there are possible unadressed qn 
    ### with the correct flux
    
    # the flux of Q1 should be zero through the construction
    # therefore we need just to find possible blocks
    # which would be probl. available in the full (qindl,dag(qindl))
    # matrix with 0 flux but are not present in the Q1 matrix
    # these have to be filled in the Q2 matrix
    
    imgsp = space(qindl)
    q1sp  = space(qind1r)
    q1q   = qn.(q1sp)
    q1d   = blockdim.(q1sp)
    
    orthosp = Vector{Pair{QN,Int}}()
    
    # imgsp is a Vector{Pair{QN, Int}}, such that
    # we can unpack it to get the qn and dimension directly
    exists_in_both = Vector{Int}() # list of tracking blocks existing in both tensors
    for (spq, spd) in imgsp
        # now test it the qn is present in the q1q array
        # holding all target qn numbers of the Q1 tensor
        if(spq in q1q)
            idx = findfirst(x -> x == spq, q1q)
            # check if it has the full dimensionality
            if(q1d[idx] < spd)
                push!(orthosp, spq => spd - q1d[idx])
                push!(exists_in_both,length(orthosp))
            end
        else
            push!(orthosp, spq => spd)
        end
    end
    
    # generating the index for the orthogonal space
    qortags = addtags(tags(qind1r), "ortho")
    qind2r  = Index(orthosp, qortags; dir = dir(qind1r))
    
    # savety check, the dimensionality should add up
    # to the total dimensionality of the left leg
    @assert dim(qind1r) + dim(qind2r) == dim(qindl)
    
    indsQ2 = setindex(inds(T), qind2r, 2)
    
    # now the different blocks of the Q2 tensor have to be created
    # their size has to be the same as the number of blocks in the
    # right Index
    nzblocksQ2 = Vector{Block{2}}(undef, nblocks(qind2r))
    
    # go through all possible targets
    for (jj, img) in enumerate(orthosp)
        imgq = qn(img)
        blockQ2L = ITensors.block(first, qindl, imgq)
        blockQ2R = ITensors.block(first, qind2r, dir(qind2r)*imgq)
        nzblocksQ2[jj] = (blockQ2L,blockQ2R)
    end
    
    # now build the Q2 tensor
    Q2 = BlockSparseTensor(ElT, undef, nzblocksQ2, indsQ2)
   
    # now looping through all blocks in the Q2 matrix
    for (jj, blockQ2) in enumerate(nzblocksQ2)
        if(jj in exists_in_both)
            qr = qn(qind2r, blockQ2[2]) # qn of the block
            # getting the block for this qn in the qind1r index
            blockQ1R = ITensors.block(first, qind1r, dir(qind1r)*qr)
            # getting the original index for that transition
            # this encodes the index in the list of dense qr decompositions
            idx = Int(blockQ1R[1])
            Q2b = Q2s[idx]
            
        else    
            blkdim = blockdim(orthosp[jj])
            Q2b = Matrix{ElT}(I,blkdim, blkdim)
        end
        blockview(Q2, blockQ2) .= Q2b
    end
    
    return Q1,Q2,R
end



function qr_full_qn(A::ITensor, Linds...; kwargs...)
    tags::TagSet = get(kwargs, :tags, "Link,qr")
    
    Lis = commoninds(A, IndexSet(Linds...))
    Ris = uniqueinds(A, Lis)
    
    if length(Lis) == 0 && length(Ris) == 0
        error(
                "In `qr`, the left and right indices are empty (the indices of `A` are ($(inds(A))), but the input indices are ($Lis)). For now, this is not supported. You may have accidentally input the wrong indices.",
            )
    end
    
    
    CL = combiner(Lis...)
    CR = combiner(Ris...)
    
    AC = A * CR * CL
    
    if(length(Lis) == 0)
        At = tensor(AC)
        # build trivial leg
        qtriv = zero(qn(inds(AC)[1], Block(1)))
        qidx  = Index(qtriv => 1; tags = tags, dir = ITensors.In)
        Q1    = delta(qidx)
        
        nzblocksR = Vector{Block{2}}(undef, nnzblocks(AC))
        for (jj,b) in enumerate(eachnzblock(At))
            blockR = (UInt(1), b[1])
            nzblocksR[jj] = blockR
        end
        Rt = BlockSparseTensor(eltype(At), undef, nzblocksR, (dag(qidx),inds(AC)...))
        for (jj,b) in enumerate(eachnzblock(At))
            ACB = blockview(At,b)
            if VERSION < v"1.5"
                # In v1.3 and v1.4 of Julia, Ub has
                # a very complicated view wrapper that
                # can't be handled efficiently
                ACB = copy(ACB)
            end 
            ACB = reshape(ACB,(1,dim(ACB)))
            blockR = nzblocksR[jj]
            blockview(Rt, blockR) .= ACB
        end
        RC = itensor(Rt)
        
        R = RC*dag(CR)
        return Q1, ITensor(0), R, qidx, nothing
    elseif(length(Ris) == 0)
        QRT = qr_full(tensor(AC); kwargs...)
    else
        cL = combinedind(CL)
        cR = combinedind(CR)
    
        if inds(AC) != IndexSet(cL, cR)
            AC = permute(AC, cL, cR)
        end
    
        QRT = qr_full(tensor(AC); kwargs...)
    end
    
    if isnothing(QRT)
        return nothing
    end
    
    Q1T, Q2T, RT = QRT
    Q1C, RC = itensor(Q1T), itensor(RT)
    
    if(Q2T == nothing)
        Q2 = ITensor(0)
    else
        Q2C = itensor(Q2T)
    end
    
    qidx  = commonind(Q1C,RC)
    lidx  = uniqueinds(Q1C,qidx)
    if(!(Q2T == nothing))
        qoidx = uniqueinds(Q2C,lidx)
    end
    
    Q1 = Q1C * dag(CL)
    if(!(Q2T == nothing))
        Q2 = Q2C * dag(CL)
    end
    R  = RC  * dag(CR)
    
    
    settags!(Q1, tags, qidx)
    settags!(R,  tags, qidx)
    qidx = settags(qidx, tags)
    if(!(Q2T == nothing))
        tags = ITensors.addtags(tags,"ortho")
        settags!(Q2,tags,qoidx)
        return Q1, Q2, R, qidx, qoidx
    end
    return Q1, Q2, R, qidx, nothing
end

qr_full_qn (generic function with 1 method)

### Combining the sparse/dense versions

In [9]:
qr_full(A::ITensor; kwargs...) = error(ITensors.noinds_error_message("qr"))

function qr_full(A::ITensor, Linds...; kwargs...)
    if(hasqns(A))
        return qr_full_qn(A, Linds...;kwargs...)
    else
        return qr_full_dense(A,Linds...;kwargs...)
    end
end

qr_full (generic function with 6 methods)

### Testing of the dense version

In [20]:
j1 = Index(4,"j=1")
j2 = Index(4,"j=2")
s  = Index(2,"Site")
T  = randomITensor(j1,s,j2)

Q1, Q2, R, qidx,qoidx = qr_full(T, (j1,s))
println(norm(Q1*dag((Q2))))
println(Q1*R ≈ T)

3.972945262208935e-16
true


### Implementation of the variance calculated using Hubigs scheme

In [21]:
function norm2(A::ITensor)
    return real(scalar(A*dag(A)))
end

function calc_var(ψ, H; full_var = false)
    if(full_var)
        H2 = abs(inner(H,ψ,H,ψ))
        E  = inner(ψ,H,ψ)
        return H2 - E^2
    end
    
    # copying the data
    ψ_cp = copy(ψ)
    orthogonalize!(ψ_cp,1)
    nrm = norm(ψ_cp)
    ψ_cp[1] = ψ_cp[1]/nrm
    
    L = length(ψ_cp)
    
    XF = Vector{ITensor}(undef,L-1)
    
    #println(ψ_cp[1])
    # building the first left environment tensor
    Li = ITensor(1)
    Ri = ITensor(1)
    
    
    XF   = Vector{ITensor}(undef, L)
    #XF[1]   = ITensor(1)
    rinds = commoninds(ψ_cp[1],ψ_cp[2])
    linds = uniqueinds(ψ_cp[1],rinds)
    for jj = 1:L - 1
        T = ψ_cp[jj]
        
        A,F,R,qidx,qoidx = qr_full(T, linds; tags = "Link,l=$(jj)")
    
        ψ_cp[jj]    = A
        ψ_cp[jj+1] *= R
        
        ITensors.setleftlim!(ψ_cp,jj)
        ITensors.setrightlim!(ψ_cp,jj+2)
        
        XF[jj] = ((Li * A)*H[jj])
        #XF[jj+1] = ((XF[jj] * A)*H[jj])*dag(prime(A))

        Li = XF[jj]*dag(prime(A))
        XF[jj]   *= dag(prime(F))
        
        linds = (qidx,inds(ψ_cp[jj+1],"Site")...)
    end
    A,F,R,qidx,qoidx = qr_full(ψ_cp[L], linds; tags = "Link,l=$L")
    XF[L] = ((Li*ψ_cp[L])*H[L])*dag(prime(F))
    res = norm2(XF[L])
    #res = 0
    
    linds = commoninds(ψ_cp[L], ψ_cp[L - 1])
    rinds = uniqueinds(ψ_cp[L], linds)
    R_scale = ITensor(1)
    for jj = L:-1:2
        T = ψ_cp[jj]
        B,G,R,qidx,qoidx = qr_full(T, rinds; tags = "Link,l=$(jj - 1)")
        #B,R, qidx = qr_small(T, rinds; tags = "Link,l=$(jj-1)")

        
        ψ_cp[jj]      = B
        ψ_cp[jj - 1] *= R
        ITensors.setleftlim!(ψ_cp,jj - 2)
        ITensors.setrightlim!(ψ_cp,jj)
        
        ###########################################################################################
        #display(XF[jj]*T*H[jj])
        #(((XF[jj]*T)*H[jj])*dag(prime(T)) * Ri) |> scalar |> display
        #Ri = ((Ri*B)*H[jj])*dag(prime(B))        
        
        #rinds = (qidx, inds(ψ_cp[jj-1], "Site")...)
        ###########################################################################################
        
        YF = ((Ri*T)*H[jj])*dag(prime(G)) 
        XF[jj-1]*YF |> norm2 |> x -> (res += x) #|> display
        
        
        if(jj < L)
            (XF[jj]*R_scale)*Ri |> norm2 |> x -> (res += x) #|> display
        end
        R_scale = R
        Ri = ((Ri*B)*H[jj])*dag(prime(B))
        rinds = (qidx, inds(ψ_cp[jj-1], "Site")...)
    end
    
    
    ###############################################################################
    return res
    
end

calc_var (generic function with 1 method)

## Do some testing of the variance on a dens mps

In [53]:

N = 60
sites = siteinds("S=1/2",N);
psi0 = randomMPS(sites,1);
ampo = OpSum()
for j=1:N-1
    ampo += 0.5,"S+",j,"S-",j+1
    ampo += 0.5,"S-",j,"S+",j+1
    ampo += "Sz",j,"Sz",j+1
end
H = MPO(ampo,sites);
sweeps = Sweeps(5) # number of sweeps is 5
maxdim!(sweeps,10,20,100,100,100,200) # gradually increase states kept
    
cutoff!(sweeps,1E-10) # desired truncation error
#psi0 = copy(psi)
full_var_0  = calc_var(psi0,H;full_var = true)
println("Full variance before dmrg: ",full_var_0)
smart_var_0 = calc_var(psi0,H;full_var = false)
println("Smart variance before dmrg: ",smart_var_0)
println("Difference: ", abs(full_var_0 - smart_var_0))
    
energy,psi = dmrg(H,psi0,sweeps);
    
full_var_1  = calc_var(psi,H;full_var = true)
println("Full variance after dmrg: ",full_var_1)
smart_var_1 = calc_var(psi,H;full_var = false)
println("Smart variance after dmrg: ",smart_var_1)
println("Difference: ", abs(full_var_1 - smart_var_1))
sz = expect(psi, "Sz");


Full variance before dmrg: 10.15859368932418
Smart variance before dmrg: 10.156528928972152
Difference: 0.002064760352027406
After sweep 1 energy=-25.894126119972 maxlinkdim=4 maxerr=1.55E-15 time=0.043
After sweep 2 energy=-26.392955115996 maxlinkdim=16 maxerr=4.50E-16 time=0.082
After sweep 3 energy=-26.402989431847 maxlinkdim=55 maxerr=9.85E-11 time=0.258
After sweep 4 energy=-26.403015091356 maxlinkdim=76 maxerr=1.00E-10 time=0.563
After sweep 5 energy=-26.403015129871 maxlinkdim=77 maxerr=9.99E-11 time=0.682
Full variance after dmrg: 6.131722329882905e-8
Smart variance after dmrg: 6.132357022516837e-8
Difference: 6.3469263393232186e-12


In [54]:
@btime calc_var(psi,H;full_var = true)
@btime calc_var(psi,H;full_var = false)

  180.025 ms (28483 allocations: 498.46 MiB)
  115.259 ms (57514 allocations: 327.19 MiB)


6.132357022516837e-8

In [31]:
orthogonalize(psi,N÷2)
T = psi[N÷2]
idxT = inds(T)

((dim=82|id=383|"Link,l=30"), (dim=2|id=968|"S=1/2,Site,n=30"), (dim=75|id=376|"Link,l=29"))

In [34]:
@btime qr(T,idxT[1:2]);

  385.389 μs (131 allocations: 436.33 KiB)


In [35]:
@btime qr_full(T,idxT[1:2]);

  509.870 μs (180 allocations: 935.25 KiB)


### Now try to do a qn qr decomposition

In [38]:
# constructing the qn of the left/right leg
qns = Vector{QN}(undef, 9)
for jj in eachindex(qns)
    qns[jj] = QN("N", jj - 1)
end
# creating a left index with qn 0 and 1 and 2, 0 being 1 dim, 1 being 2 dim and 2 being 2 dim
j_l  = dag(Index([qns[1] => 1, qns[2] => 2], "l=1"));
# creating a site index with 0 and 1 qn both being 1 dim
si   = dag(Index([qns[1] => 1, qns[2] =>1], "Site";));
# creating a right index with the same qn as the left index, but different dimensions
j_r = Index([qns[1] => 2, qns[2] => 4, qns[3] => 4], "l=2,1");
# creating a right index with the same qn as the combination of j_l and si
#j_r2 = Index()

In [39]:
T2s = randomITensor(j_r,j_l)
T3s = randomITensor(j_l, si, j_r);

In [43]:
Q,R,qidx = qr_small(T3s, j_r, si);
println(Q*R ≈ T3s)

true


In [44]:
Q1,Q2,R,qidx,qoidx = qr_full(T3s, j_r, si);
println(Q1*R ≈ T3s)
println(norm(Q1*dag(Q2)))

true
2.186852714097718e-16


In [45]:
@btime qr_small(T3s,j_r,si);
@btime qr_full(T3s, j_r,si)
@btime factorize(T3s,j_r,si);

  73.675 μs (875 allocations: 124.48 KiB)
  98.732 μs (1221 allocations: 180.50 KiB)
  122.906 μs (1235 allocations: 177.94 KiB)


In [50]:
N = 60
sites = siteinds("S=1/2",N; conserve_qns = true);
#states = [isodd(jj) ? "Up" : "Dn" for jj in 1:N]
states = ["Up" for jj in 1:N]
states[N÷2] = "Dn"
psi0 = randomMPS(sites, states,1);
ampo = OpSum()
for j=1:N-1
    ampo += 0.5,"S+",j,"S-",j+1
    ampo += 0.5,"S-",j,"S+",j+1
    ampo += "Sz",j,"Sz",j+1
end
H = MPO(ampo,sites);
sweeps = Sweeps(5) # number of sweeps is 5
maxdim!(sweeps,10,20,100,100,100,200) # gradually increase states kept

cutoff!(sweeps,1E-10) # desired truncation error

full_var_0  = calc_var(psi0,H;full_var = true)
println("Full variance before dmrg: ",full_var_0)
smart_var_0 = calc_var(psi0,H;full_var = false)
println("Smart variance before dmrg: ",smart_var_0)
println("Difference: ", abs(full_var_0 - smart_var_0))
    
energy,psi = dmrg(H,psi0,sweeps);
   
full_var_1  = calc_var(psi,H;full_var = true)
println("Full variance after dmrg: ",full_var_1)
smart_var_1 = calc_var(psi,H;full_var = false)
println("Smart variance after dmrg: ",smart_var_1)
println("Difference: ", abs(full_var_1 - smart_var_1))

Full variance before dmrg: 0.5
Smart variance before dmrg: 0.5
Difference: 0.0
After sweep 1 energy=12.755938432009 maxlinkdim=2 maxerr=9.70E-11 time=8.335
After sweep 2 energy=12.753423845688 maxlinkdim=2 maxerr=9.70E-11 time=0.084
After sweep 3 energy=12.752428313906 maxlinkdim=2 maxerr=1.39E-17 time=0.082
After sweep 4 energy=12.751846156467 maxlinkdim=2 maxerr=1.39E-17 time=0.083
After sweep 5 energy=12.751550866698 maxlinkdim=2 maxerr=6.94E-18 time=0.083
Full variance after dmrg: 1.1098509560270031e-6
Smart variance after dmrg: 1.1098508535422829e-6
Difference: 1.0248472026802655e-13


In [51]:
@btime calc_var(psi,H;full_var = true)
@btime calc_var(psi,H;full_var = false)

  7.117 ms (111032 allocations: 14.03 MiB)
  21.056 ms (222834 allocations: 28.20 MiB)


1.1098508535422829e-6